In [476]:
import pandas as pd 
import glob # a module that finds all the pathnames matching a specified pattern
import numpy as np

In [16]:
file_path =r'/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects'

In [18]:
all_csv_Files = glob.glob(file_path + "/*.csv")
all_excel_Files = glob.glob(file_path + "/*.xlsx")
print(all_csv_Files)
print(all_excel_Files)

['/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects/HS.csv', '/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects/HS2.csv']
['/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects/HS2.xlsx']


In [23]:
for file in all_csv_Files:
    df = pd.read_csv(file,encoding='latin1')
    print(df.head())
    print('\n')
# the data are not meesy in the csv files

  Geographic Area           City percent_completed_hs
0              AL  Anniston city                 79.8
1              AL      Arab city                 87.6
2              AL   Ardmore town                 77.7
3              AL      Argo town                 87.5
4              AL    Ariton town                 71.3


  Geographic Area           City percent_completed_hs
0              AL  Anniston city                 79.8
1              AL      Arab city                 87.6
2              AL   Ardmore town                 77.7
3              AL      Argo town                 87.5
4              AL    Ariton town                 71.3




In [33]:
for file in all_excel_Files:
    df = pd.read_excel(file)
    print(df.head())
    print('\n')
    
# the data in excel files are pretty messy. a lot of null values and the column names are wrong.

   Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3       Unnamed: 4  \
0         NaN         NaN         NaN         NaN              NaN   
1         NaN         NaN         NaN         NaN              NaN   
2         NaN         NaN         NaN         NaN              NaN   
3         NaN         NaN         NaN         NaN  Geographic Area   
4         NaN         NaN         NaN         NaN               AL   

      Unnamed: 5            Unnamed: 6  
0            NaN                   NaN  
1            NaN                   NaN  
2            NaN                   NaN  
3           City  percent_completed_hs  
4  Anniston city                  79.8  




# For convenience's sake, I will just work on the xlsx file. 


In [726]:
# for single file
df = pd.read_excel('/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects/HS2.xlsx')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Geographic Area,City,percent_completed_hs
4,NaN,NaN,NaN,NaN,AL,Anniston city,79.8


In [727]:
#drop the columns and rows that contain all null values
df.dropna(how='all',axis=1, inplace=True)
df.dropna(how='all',axis=0, inplace=True)
df.head()

,Unnamed: 4,Unnamed: 5,Unnamed: 6
3,Geographic Area,City,percent_completed_hs
4,AL,Anniston city,79.8
5,AL,Arab city,87.6
6,AL,Ardmore town,77.7
7,AL,Argo town,87.5


In [728]:
# set the first row as the column names
# using .values to prevent extracting the index 3
header= df.iloc[0].values 

# remove the first row since it will be used as the column names
df = df[1:]

# set the new column names
df.columns = header

# reset the idex and drop the original one.
df=df.reset_index(drop=True)
df.head()

,Geographic Area,City,percent_completed_hs
0,AL,Anniston city,79.8
1,AL,Arab city,87.6
2,AL,Ardmore town,77.7
3,AL,Argo town,87.5
4,AL,Ariton town,71.3


In [729]:
# check to see if there are missing values in either of the columns.
df.isnull().sum()

Geographic Area         0
City                    0
percent_completed_hs    0
dtype: int64

In [739]:
df.info()

# the data type for percent_completed_hs shold be float type, not str

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29315 entries, 0 to 29314
Data columns (total 3 columns):
Geographic Area         29315 non-null object
City                    29315 non-null object
percent_completed_hs    29315 non-null object
dtypes: object(3)
memory usage: 687.1+ KB


In [741]:
# remove the special char in each value. if the value is empty or any other char, replace it with empty str
# only keep values that between 0-9 and '.'.
import re
for i in range(len(df['percent_completed_hs'])):
    df['percent_completed_hs'][i] = re.sub('[^0-9.]','', df['percent_completed_hs'][i])

In [742]:
df.head()

,Geographic Area,City,percent_completed_hs
0,AL,Anniston city,79.8
1,AL,Arab city,87.6
2,AL,Ardmore town,77.7
3,AL,Argo town,87.5
4,AL,Ariton town,71.3


In [743]:
# After the above code, values that do not contain numbers but others should be just an empty cell now. 
# but they could be converted into float. 
a=0
for i in df['percent_completed_hs']:
    if i=='':
        a+=1
print(a)
a=0

# 197 values with an empty str

197


In [744]:
# Set all those empty values to 0
for i in range(len(df['percent_completed_hs'])):
               if df['percent_completed_hs'][i]=='':
                    df['percent_completed_hs'][i] = 0

In [745]:
# convert the column to float data type
df['percent_completed_hs'] = [float(x) for x in df['percent_completed_hs']]

In [747]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29315 entries, 0 to 29314
Data columns (total 3 columns):
Geographic Area         29315 non-null object
City                    29315 non-null object
percent_completed_hs    29315 non-null float64
dtypes: float64(1), object(2)
memory usage: 687.1+ KB


In [758]:
df.isnull().sum()

Geographic Area         0
City                    0
percent_completed_hs    0
dtype: int64

In [823]:
# establish the connect with MySQL
import pymysql
conn = pymysql.connect(host='localhost', user='root', password='1125', db='just trying') 
cursor = conn.cursor()

In [824]:
# get all the column name from the database

query = 'SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA="just trying" AND TABLE_NAME="frompy";'
cursor.execute(query)
data = cursor.fetchall()
for i in data:
    print(*i, end=' ')
    
# now you can just copy the column names and use it.

id Geographic Area City percent_completed_hs 

In [790]:
query = 'insert into frompy (`Geographic Area`,City,percent_completed_hs);'
for i in df2:
    cursor.execute('insert into frompy (`Geographic Area`, City, percent_completed_hs)' 'values("%s", "%s", "%s")', [*i])
conn.commit()


# it worked. :)

In [828]:
query = 'select * from frompy;'
cursor.execute(query)
data = cursor.fetchall()
a=0
for i in data:
    print(*i)
    a+=1
    if(a==10):
        break

1 'AL' 'Anniston city' 79.8
2 'AL' 'Arab city' 87.6
3 'AL' 'Ardmore town' 77.7
4 'AL' 'Argo town' 87.5
5 'AL' 'Ariton town' 71.3
6 'AL' 'Arley town' 80.6
7 'AL' 'Ashford town' 83.5
8 'AL' 'Ashland town' 77.9
9 'AL' 'Ashville city' 77.6
10 'AL' 'Athens city' 83.4


In [820]:
cursor.close()
conn.close()

In [ ]:
# data are successfully loaded into the database. 
# the remain problem is that the name of the states and cities contain the str '', for example, 'NY'. 

# In a nutshell

In [840]:
file_path =r'/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects'
all_excel_Files = glob.glob(file_path + "/*.xlsx")
print(all_excel_Files)

query = 'insert into frompy (`Geographic Area`,City,percent_completed_hs);'
 
conn = pymysql.connect(host='localhost', user='root', password='1125', db='just trying') 
cursor = conn.cursor() 


['/Users/michaelzeng/Library/Mobile Documents/com~apple~CloudDocs/python projects/HS2.xlsx']


In [841]:
for file in all_excel_Files:
    print('Display the data')
    df = pd.read_excel(file,encoding='latin1')
    print(df.head())
    print('\n')

    # drop columns and rows that contain only null values
    print('Drop the columns and rows contain entirely null values')
    df = df.dropna(how='all', axis=1)
    df = df.dropna(how='all', axis=0)
    print(df.head())
    print('\n')
    
    # the first row contains the name of each column
    print('Set the first row as the colmns name and then reset the index')
    header= df.iloc[0].values 
    
    # Since the first row will be used as the column name, so remove them.
    df = df[1:]
    
    # set the new column name
    df.columns = header
    
    # reset the index
    df = df.reset_index(drop=True)
    print(df.head())
    print('\n')
        
    print('Remove everything except 0-9 and "." ')
    for i in range(len(df['percent_completed_hs'])):
        df['percent_completed_hs'][i] = re.sub('[^0-9.]','', df['percent_completed_hs'][i])
    print('\n')

    print('Replace the empty srtrings with 0')
    for i in range(len(df['percent_completed_hs'])):
               if df['percent_completed_hs'][i]=='':
                    df['percent_completed_hs'][i] = 0
    print('\n')  
    
    print('Convert the column to float data type')
    df['percent_completed_hs'] = [float(x) for x in df['percent_completed_hs']]
    print('\n')  
    
    print(df.info())
    print('\n')  

    print('Start loading the data into MySQl database')  
    try:
        for i in df2:
            cursor.execute('insert into frompy (`Geographic Area`, City, percent_completed_hs)' 'values("%s", "%s", "%s")', [*i])
        conn.commit()
        print('Data loaded successfully')
        print('\n')  
    except:
        print('An error occured.')
        print('\n')  
    
    print('\n')  
    print('Connection now terminated.')
    cursor.close()
    conn.close()

Display the data
   Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3       Unnamed: 4  \
0         NaN         NaN         NaN         NaN              NaN   
1         NaN         NaN         NaN         NaN              NaN   
2         NaN         NaN         NaN         NaN              NaN   
3         NaN         NaN         NaN         NaN  Geographic Area   
4         NaN         NaN         NaN         NaN               AL   

      Unnamed: 5            Unnamed: 6  
0            NaN                   NaN  
1            NaN                   NaN  
2            NaN                   NaN  
3           City  percent_completed_hs  
4  Anniston city                  79.8  


Drop the columns and rows contain entirely null values
        Unnamed: 4     Unnamed: 5            Unnamed: 6
3  Geographic Area           City  percent_completed_hs
4               AL  Anniston city                  79.8
5               AL      Arab city                  87.6
6               AL   Ardmore town  

In [ ]:
# Nice. :)

'AL', 'Gu-Win town',87.2
'AL', 'Hackleburg town',76.5

need to find a way to remove the ''. 